In [1]:
! pip install transformers
! pip install torch
! pip install numpy
! pip install sklearn
! pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 32.9 MB/s 
     |████████████████████████████████| 163 kB 68.8 MB/s 
     |████████████████████████████████| 7.6 MB 54.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=0fa560a7b946a9d2cb25296098135950b3a7ef83eb6a9c5a1b6d1605ec6d8b8a
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [3]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [4]:
import numpy as np
import pandas as pd

path_to_train = "train_df_imbalanced.csv"

In [5]:
data = pd.read_csv(path_to_train)
data

,Unnamed: 0,productAsin,ratingScore,reviewTitle,reviewReaction,reviewDescription,isVerified,category,languages,concat_review,polarity
0,9406,B08NLXR9V5,4,Girl Alone,One person found this helpful,"My first Blake Pierce book was enjoyable, fast...",True,mystery,Language.ENGLISH,Girl Alone. My first Blake Pierce book was enj...,1
1,5701,670062510,5,A great book for young people. It has a great...,NaN,My favorite book when I was young. I read it ...,True,children,Language.ENGLISH,A great book for young people. It has a great...,1
2,13621,1542046599,1,More gripes than hypes,One person found this helpful,This is the first book I‚Äôve read by this aut...,True,mystery,Language.ENGLISH,More gripes than hypes. This is the first book...,-1
3,5021,399226907,5,Add this book to your collection,NaN,Cute and educational book to teach counting an...,True,children,Language.ENGLISH,Add this book to your collection. Cute and edu...,1
4,21354,125030170X,2,Just okay.,NaN,This is one of those books you can read in a c...,False,children,Language.ENGLISH,Just okay.. This is one of those books you can...,0
...,...,...,...,...,...,...,...,...,...,...,...
20956,12474,1542046599,5,Great read; couldn‚Äôt put it down,NaN,The first chapter reads from an odd point of v...,True,mystery,Language.ENGLISH,Great read; couldn‚Äôt put it down. The first ...,1
20957,22518,125030170X,1,Used books?,NaN,They are purchased as new and I received the o...,True,children,Language.ENGLISH,Used books?. They are purchased as new and I r...,-1
20958,5670,670062510,4,My 7th graders not only love this book but the...,NaN,My 7th graders not only love this book but the...,True,children,Language.ENGLISH,My 7th graders not only love this book but the...,1
20959,891,B01IW9TM5O,5,One of my favorite books ever - read almost al...,One person found this helpful,One of my favorite books ever - read almost al...,True,humor_entertainment,Language.ENGLISH,One of my favorite books ever - read almost al...,1


In [6]:
# sparse categorical cross entropy allowed range [0,2)
data.loc[data['polarity'] == 0, 'polarity'] = 0.5
data.loc[data['polarity'] == -1, 'polarity'] = 0

In [7]:
data['polarity'].value_counts()

1.0    13246
0.0     7518
0.5      197
Name: polarity, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
X = data.concat_review
y = data.polarity
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
train = pd.concat([X_train, y_train], axis=1)
train

,concat_review,polarity
11148,Terrible results. This book only loaded 17%. C...,0.0
5642,Did we read the same book..?. I feel like a lo...,0.0
17064,Strong Writing. I was surprised by this book. ...,1.0
1373,Smaller than I thought it would be. Cool book ...,0.0
19310,Amazing!!!. I could not put this book down.I f...,1.0
...,...,...
11284,A Christina Lauren story like no other. Christ...,1.0
11964,Great. Really quick read. Great page Turner,1.0
5390,Baby‚Äôs favorite. My baby‚Äôs No.1 favorite b...,1.0
860,Wonderful!. I very much enjoyed this book. The...,1.0


In [10]:
train['polarity'].value_counts()

1.0    10616
0.0     5993
0.5      159
Name: polarity, dtype: int64

In [11]:
test = pd.concat([X_test, y_test], axis=1)
test

,concat_review,polarity
17089,Good book. This is a really cute book. Grandda...,1.0
1182,Emotional. I flew through this book unable to ...,1.0
11369,Size matters. I remember reading this book as ...,1.0
1427,Good story. I thoroughly enjoyed reading this ...,1.0
10319,Bait and switch. Hated this book. It is not a ...,0.0
...,...,...
6535,Loved it. Blake Pierce is an EXCEPTIONAL write...,1.0
14706,Terrible Self Help Book. This book is the ramb...,0.0
11842,SOOO CUTE. I loved this book finished it in on...,1.0
1303,Relevant for the times. In an age where televi...,0.0


In [12]:
test['polarity'].value_counts()

1.0    2630
0.0    1525
0.5      38
Name: polarity, dtype: int64

In [13]:
DATA_COLUMN = 'concat_review'
LABEL_COLUMN = 'polarity'

def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

In [14]:
import tensorflow as tf

def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [15]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [17]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
1048/1048 [==============================] - 896s 832ms/step - loss: 0.1034 - accuracy: 0.9631 - val_loss: 0.2198 - val_accuracy: 0.9301
Epoch 2/2
1048/1048 [==============================] - 868s 828ms/step - loss: 0.0307 - accuracy: 0.9908 - val_loss: 0.2906 - val_accuracy: 0.9337


In [18]:
pred_sentences = ["Awful. I gave up after 38% of my Kindle. Yes we were told someone had died but we don't know who.All the characters with the possible exception of the wedding planner and her husband were unpleasant. It was like a poor copy of 'And then there were none'",
                  "another easy read. The author Jasmine Guillory hit the mark with her books. This was funny warm and easy reading. This was the second of her books that I have read. Enjoy",
                  "Anti-climatic. Definitely held my interest throughout, but I feel as though the story ended very suddenly. And I was left wondering what came of one of the main characters.... her story was unfinished."]

In [32]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Neutral', 'Positive']
for i in range(len(tf_predictions)):
  if tf_predictions[i][1] >= 0.0 and tf_predictions[i][1] < 0.3:
    label = 'Negative'
  elif tf_predictions[i][1] >= 0.3 and tf_predictions[i][1] < 0.6:
    label = 'Neutral'
  else:
    label = 'Positive'
  print(pred_sentences[i], ": \n", label, tf.print(tf_predictions[i][1]))

0.000553067948
Awful. I gave up after 38% of my Kindle. Yes we were told someone had died but we don't know who.All the characters with the possible exception of the wedding planner and her husband were unpleasant. It was like a poor copy of 'And then there were none' : 
 Negative None
0.999767601
another easy read. The author Jasmine Guillory hit the mark with her books. This was funny warm and easy reading. This was the second of her books that I have read. Enjoy : 
 Positive None
0.00276046782
Anti-climatic. Definitely held my interest throughout, but I feel as though the story ended very suddenly. And I was left wondering what came of one of the main characters.... her story was unfinished. : 
 Negative None
